In [10]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import mean_squared_error

import mlflow

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import os

import pandas as pd

In [5]:
df = pd.read_parquet("../data/df.parquet")
train_df = pd.read_parquet("../data/train_df.parquet")
val_df = pd.read_parquet("../data/val_df.parquet")
test_df = pd.read_parquet("../data/test_df.parquet")

In [6]:
y = "Methyl tert-butyl ether (MTBE)"

In [7]:
X_col = [c for c in df.columns if c not in [y, "sample_date", "station_id"]]

In [8]:
dv = DictVectorizer()

train_dicts = train_df[X_col].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = val_df[X_col].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_train = train_df[y].values
y_val = val_df[y].values

In [9]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [11]:
os.environ["AWS_PROFILE"] = "default"
TRACKING_SERVER_HOST= "ec2-54-147-5-224.compute-1.amazonaws.com"

In [13]:
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
mlflow.set_experiment("water-quality-prediction")

2023/08/05 06:22:06 INFO mlflow.tracking.fluent: Experiment with name 'water-quality-prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlops-zoomcamp-2023/project/2', creation_time=1691216526901, experiment_id='2', last_update_time=1691216526901, lifecycle_stage='active', name='water-quality-prediction', tags={}>

In [14]:
mlflow.xgboost.autolog()

In [15]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")     

    return {'loss': rmse, 'status': STATUS_OK}

In [16]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[06:23:57] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.53009                           
[1]	validation-rmse:0.98210                           
[2]	validation-rmse:1.21251                           
[3]	validation-rmse:1.23930                           
[4]	validation-rmse:1.24345                           
[5]	validation-rmse:1.28766                           
[6]	validation-rmse:1.31533                           
[7]	validation-rmse:1.33545                           
[8]	validation-rmse:1.37611                           
[9]	validation-rmse:1.38569                           
[10]	validation-rmse:1.35702                          
[11]	validation-rmse:1.35886                          
[12]	validation-rmse:1.37719                          
[13]	validation-rmse:1.36822                          
[14]	validation-rmse:1.34788                          
[15]	validation-rmse:1.34770                          
[

2023/08/05 06:24:33 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.



[06:24:36] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.49430                                                     
[1]	validation-rmse:0.57869                                                     
[2]	validation-rmse:0.61559                                                     
[3]	validation-rmse:0.85166                                                     
[4]	validation-rmse:0.87731                                                     
[5]	validation-rmse:0.98317                                                     
[6]	validation-rmse:0.99677                                                     
[7]	validation-rmse:1.01095                                                     
[8]	validation-rmse:1.09928                                                     
[9]	validation-rmse:1.10889                                                     
[10]	validation-rmse:1.12577                                           

2023/08/05 06:25:04 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2023/08/05 06:25:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."

2023/08/05 06:25:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site

[06:25:06] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.51958                                                     
[1]	validation-rmse:0.57338                                                     
[2]	validation-rmse:0.64458                                                     
[3]	validation-rmse:0.72508                                                     
[4]	validation-rmse:0.81006                                                     
[5]	validation-rmse:0.89516                                                     
[6]	validation-rmse:0.96967                                                     
[7]	validation-rmse:1.14024                                                     
[8]	validation-rmse:1.20942                                                     
[9]	validation-rmse:1.39977                                                     
[10]	validation-rmse:1.45152                                           

...

[06:46:22] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.49432                                                      
[1]	validation-rmse:0.49891                                                      
[2]	validation-rmse:0.61200                                                      
[3]	validation-rmse:0.63073                                                      
[4]	validation-rmse:0.78152                                                      
[5]	validation-rmse:0.94630                                                      
[6]	validation-rmse:0.96437                                                      
[7]	validation-rmse:1.06568                                                      
[8]	validation-rmse:1.21431                                                      
[9]	validation-rmse:1.23361                                                      
[10]	validation-rmse:1.32865                                 

2023/08/05 06:46:48 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2023/08/05 06:46:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."

2023/08/05 06:46:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site

[06:46:51] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.55416                                                      
[1]	validation-rmse:0.65164                                                      
[2]	validation-rmse:0.73223                                                      
[3]	validation-rmse:0.69432                                                      
[4]	validation-rmse:0.75173                                                      
[5]	validation-rmse:0.77709                                                      
[6]	validation-rmse:0.82990                                                      
[7]	validation-rmse:0.87656                                                      
[8]	validation-rmse:0.89763                                                      
[9]	validation-rmse:0.90442                                                      
[10]	validation-rmse:0.92797                                 

2023/08/05 06:46:56 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2023/08/05 06:46:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."

2023/08/05 06:46:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site

100%|██████████| 50/50 [23:01<00:00, 27.63s/trial, best loss: 0.5934676327714429]
